## Hi all,

I'm a data analyst in Coupang which is the fastest growing e-commerce company in South Korea. I got a master degree of management engineering. Main research topic was empirical finance, which analyzes financial data with various econometrics methods. I'm familiar with cross-sectional and panel regression with fixed effects. My last project is Sales forecast with Prophet, ARIMA, and RNN. Recently, I'm interested in Deep learning, Machine learning techniques to define and solve problems in transportation systems. If you have any questions, please leave a comment or send me an e-mail. This document is still working version.

안녕하세요. 쿠팡에 Junior Software Engineer로 재직중인 윤선미입니다. 데이터 분석하는 일을 하고 있습니다. UNIST 에서 경영공학으로 석사학위를 받았고 세부전공은 계량금융입니다. Empirical finance 라고 불리는 이 분야는 금융 데이터를 여러가지 통계학적 방법론으로 분석, 해석합니다. 횡단면 자료분석, 패널 자료분석 (Cross-sectional regression and panel regression) 에 익숙합니다. 최근에는 Prophet, ARIMA, RNN을 ensemble하여 예측 문제를 푸는 프로젝트를 진행했습니다. 딥러닝, 머신러닝 방법론들을 사용하여 배송 시스템 내의 다양한 문제들을 정의하고 푸는 것에 관심이 많습니다. 코멘트, 메일을 통한 어떤 질문이라도 환영합니다. 감사합니다. 이 문서는 working 입니다. 

ysunmi0427@gmail.com, Software Engineer in Coupang, Sunmi Yoon


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))

from kaggle.competitions import twosigmanews

# Read data

In [ ]:
env = twosigmanews.make_env()
(market_train_df, news_train_df) = env.get_training_data()

In [ ]:
print('length of market_train_df: {}'.format(len(market_train_df)))
print('how many assetCode in training set: {}'.format(market_train_df['assetCode'].nunique()))
print('length of news_train_df: {}'.format(len(news_train_df)))

In [ ]:
print(market_train_df.columns)
market_train_df.head(2)

# feature translation in Korean

### Market data
이 데이터는 미국 주식시장에 상장 된 주식의 일부를 다룹니다. 데이터가 포함하는 주식은 거래량과, 정보의 사용가능 여부를 기준으로 일 단위로 바뀔 수 있습니다. Returns(수익률)은 open-to-open(시가기준), close-to-close(종가기준)으로 계산됩니다. 수익률에는 raw 타입과 Mktres 타입이 있습니다. raw 타입은 어떤 조정도 되지 않은 수익률을 뜻하고, Mktres 타입은 마켓 전반의 움직임을 제외하고 해당하는 주식의 고유한 움직임만을 포함하는 수익률을 의미합니다. (역자 주. market-residualized return = market adjusted stock return. CAPM, capital asset pricing model을 참고하세요.) 이 문제에서 수익률은 1일 또는 10일 기준으로 계산됩니다. Returns are tagged with 'Prev' if they are backwards looking in time, or 'Next' if forwards looking.

The data includes a subset of US-listed instruments. The set of included instruments changes daily and is determined based on the amount traded and the availability of information. This means that there may be instruments that enter and leave this subset of data. There may therefore be gaps in the data provided, and this does not necessarily imply that that data does not exist (those rows are likely not included due to the selection criteria).

The marketdata contains a variety of returns calculated over different timespans. All of the returns in this set of marketdata have these properties:

Returns are always calculated either open-to-open (from the opening time of one trading day to the open of another) or close-to-close (from the closing time of one trading day to the open of another).
Returns are either raw, meaning that the data is not adjusted against any benchmark, or market-residualized (Mktres), meaning that the movement of the market as a whole has been accounted for, leaving only movements inherent to the instrument.
Returns can be calculated over any arbitrary interval. Provided here are 1 day and 10 day horizons.
Returns are tagged with 'Prev' if they are backwards looking in time, or 'Next' if forwards looking.

* time: 현재 시간 / the current time (in marketdata, all rows are taken at 22:00 UTC)
* assetCode: 자산의 ID, 주식의 고유넘버. 한 회사가 여러 assetCode를 가질 수 있다. (역자 주. 아마 우선주, 보통주 등 다양한 type의 주식이 있기 때문인 것 같습니다.) / a unique id of an asset. a single company may have multiple assetCodes
* assetName: 자산의 이름, 법인명. 관련 뉴스 데이터가 하나도 없다면 'Unknown' 표시된다. / the name that corresponds to a group of assetCodes. These may be "Unknown" if the corresponding assetCode does not have any rows in the news data.
* volume: 당일 거래 물량 / trading volume in shares for the day
* close: 당일 종가 (주식 분할, 배당금 조정이 반영되지 않음) / the close price for the day (not adjusted for splits or dividends)
* open: 당일 시가 (주식 분할, 배당금 조정이 반영되지 않음) (역자 주. 전일 종가와 같지 않습니다.) / the open price for the day (not adjusted for splits or dividends)
* returnsClosePrevRaw1
* returnsOpenPrevRaw1
* returnsClosePrevRaw10
* returnsOpenPrevRaw10
* returnsClosePrevMktres10
* returnsOpenPrevMktres10
* returnsOpenNextPrevMktres10
* universe: a boolean indicating whether or not the instrument on that day will be included in scoring. This value is not provided outside of the training data time period. The trading universe on a given date is the set of instruments that are avilable for trading (the scoring function will not consider instruments that are not in the trading universe). The trading universe changes daily.

# EDA

## About market price

In [ ]:
print(market_train_df.columns)
market_train_df.head(2)

### General trends of prices
* It is cool to be able to see how markets fall and rise again through a decade. There are serious stock price drops on the market. That serious price drops were coming from collapse of Lehman Brothers during subprime mortgage crisis, Black Monday, Oil prices crash and et cetera. Should we include these big crisis in our data for forecast? Should we exclude all the big crisis for training our model?
* 10년 동안 주식 가격의 움직임을 볼 수 있다. 아래 figure는 각각 quantile 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95 지점의 종가를 날짜별로 보여준다. 심각한 주식가격 하락도 관찰할 수 있다. 특히 2008년 ~ 2009년에는 서브 프라임 모기지 사건, 르만 브라더스의 파산 등의 굵직한 사건들이 있었다. 이런 큰 사건들을 데이터에 포함시키는 것이 forecast에 도움이 될까? 큰 사건들을 데이터에서 무조건 제외시키는 것은 과연 모델 트레이닝에 도움이 될까?

In [ ]:
for i in [0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95]:
    market_train_df.groupby('time')['close'].quantile(i).plot(figsize=(10,6))

### Anomaly
* Check close and open price have right value.
* Close, Open 데이터에 문제가 없는지 체크한다. 그날의 시가와 종가가 상식 이상으로 많이 차이나는 (100% 이상) 경우는 데이터에 문제가 있다고 보았다. References에 EDA feature engineering and everything 커널을 참조했다.

In [ ]:
market_train_df['close_to_open_abs'] = (market_train_df['close'] / market_train_df['open']).abs()

In [ ]:
print('price increased by 100% or more: {}'.format(len(market_train_df[market_train_df['close_to_open_abs'] >= 2])))
print('price decreased by 100% or more: {}'.format(len(market_train_df[market_train_df['close_to_open_abs'] <= 0.5])))

In [ ]:
# 얘네를 어떤 값으로 대체를 해줘야 좋을까?
market_train_df[(market_train_df['close_to_open_abs'] >= 2) | (market_train_df['close_to_open_abs'] <= 0.5)]

## About news

In [ ]:
print(news_train_df.columns)
news_train_df.head(2)

# References
* https://www.kaggle.com/artgor/eda-feature-engineering-and-everything